In [7]:
# imports for downloading files
import urllib.request
import zipfile
import os
from PIL import Image
import glob
import cv2

#  imports for the network
import torchvision, torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time

In [3]:
# Get files in current working directory
files = os.listdir()


# This url points to the download of the .zip file for the masks
masks_url = ""

# This url points to the download of the .zip file for the masks
images_url = ""

urls =  [masks_url, images_url]

for url in urls:
    # Download the zip file
    print("Downloading zip file...")
    u = urllib.request.urlopen(url)
    data = u.read()
    u.close()

    zip_filename = ""
    if url == masks_url:
        # Specify the local filename for the downloaded zip file
        zip_filename = 'Masks.zip'
    elif url == images_url:
        # Specify the local filename for the downloaded zip file
        zip_filename = 'Images.zip'

    with open(zip_filename, 'wb') as f:
        f.write(data)

    # Unzip the downloaded file
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        # Extract all contents to the current working directory
        zip_ref.extractall()
  

ValueError: unknown url type: ''

In [62]:
class FireDataset(Dataset):

    def __init__(self, image_path,masks_path, transform):
        self.data = []
        self.image_path = image_path
        self.masks_path = masks_path
        self.img_dim=(3840, 2160)
        self.transform = transform

        self.image_file_list = glob.glob(self.image_path + "/*.jpg")
        self.masks_file_list = glob.glob(self.masks_path + "/*.png")

        for i in range(0,len(self.image_file_list)):
                
                img = cv2.imread(self.image_file_list[i])[..., ::-1]  # Convert BGR to RGB
                img = cv2.resize(img, self.img_dim)
                img_tensor = self.transform(img)

                mask = cv2.imread(self.masks_file_list[i], cv2.IMREAD_UNCHANGED)
                mask = mask * 255
                masks_path = cv2.resize(mask, self.img_dim)
                mask_tensor = self.transform(mask)

                self.data.append([img_tensor, mask_tensor])

    def __len__(self):
        return len(self.data)


    def get_data (self):

        return self.data

In [64]:
image_path = "Images"
mask_path = "Masks"

image_files = os.listdir(image_path)
mask_files = os.listdir(mask_path)

transform = transforms.Compose([transforms.ToTensor()])
dataset = FireDataset(image_path,mask_path, transform)
length = FireDataset.__len__(dataset)
print(dataset.get_data())

"""
#now we split the dataset into train(70), validation(15) and test (15)

# Split the image_mapping into train, validation, and test subsets
train_mapping, test_mapping = train_test_split(list(image_mapping.items()), test_size=0.3, random_state=42)
validation_mapping, test_mapping = train_test_split(test_mapping, test_size=0.5, random_state=42)

# Convert the train, validation, and test mappings back to dictionaries
train_mapping = dict(train_mapping)
validation_mapping = dict(validation_mapping)
test_mapping = dict(test_mapping)

# Print the train, validation, and test mappings
print("Train Mapping:")
print(train_mapping)
print("Validation Mapping:")
print(validation_mapping)
print("Test Mapping:")
print(test_mapping)

"""

[[tensor([[[0.1922, 0.1922, 0.1922,  ..., 0.1647, 0.1647, 0.1647],
         [0.1922, 0.1922, 0.1922,  ..., 0.1647, 0.1647, 0.1647],
         [0.1922, 0.1922, 0.1922,  ..., 0.1647, 0.1647, 0.1647],
         ...,
         [0.6235, 0.6235, 0.6235,  ..., 0.6314, 0.6314, 0.6353],
         [0.6235, 0.6235, 0.6235,  ..., 0.6314, 0.6314, 0.6353],
         [0.6235, 0.6235, 0.6235,  ..., 0.6314, 0.6314, 0.6353]],

        [[0.1373, 0.1373, 0.1373,  ..., 0.1804, 0.1804, 0.1804],
         [0.1373, 0.1373, 0.1373,  ..., 0.1804, 0.1804, 0.1804],
         [0.1373, 0.1373, 0.1373,  ..., 0.1804, 0.1804, 0.1804],
         ...,
         [0.6392, 0.6392, 0.6392,  ..., 0.6431, 0.6431, 0.6471],
         [0.6392, 0.6392, 0.6392,  ..., 0.6431, 0.6431, 0.6471],
         [0.6392, 0.6392, 0.6392,  ..., 0.6431, 0.6431, 0.6471]],

        [[0.0941, 0.0941, 0.0941,  ..., 0.2157, 0.2157, 0.2157],
         [0.0941, 0.0941, 0.0941,  ..., 0.2157, 0.2157, 0.2157],
         [0.0941, 0.0941, 0.0941,  ..., 0.2157, 0.2157, 

'\n#now we split the dataset into train(70), validation(15) and test (15)\n\n# Split the image_mapping into train, validation, and test subsets\ntrain_mapping, test_mapping = train_test_split(list(image_mapping.items()), test_size=0.3, random_state=42)\nvalidation_mapping, test_mapping = train_test_split(test_mapping, test_size=0.5, random_state=42)\n\n# Convert the train, validation, and test mappings back to dictionaries\ntrain_mapping = dict(train_mapping)\nvalidation_mapping = dict(validation_mapping)\ntest_mapping = dict(test_mapping)\n\n# Print the train, validation, and test mappings\nprint("Train Mapping:")\nprint(train_mapping)\nprint("Validation Mapping:")\nprint(validation_mapping)\nprint("Test Mapping:")\nprint(test_mapping)\n\n'